# Set up environment

In [1]:
# Import the 'drive' module from the 'google.colab' package to enable Google Drive integration.
# Then, mount Google Drive to the '/drive' directory within the Colab environment.
# The 'force_remount=True' parameter ensures that the Drive is remounted even if it was previously mounted.

from google.colab import drive
drive.mount('/drive', force_remount=True)

# This allows easy access to files stored in Google Drive.
%cd '/drive/MyDrive/Colab Notebooks/Projects/House Price Competition'

Mounted at /drive
/drive/MyDrive/Colab Notebooks/Projects/House Price Competition


In [2]:
import pandas as pd
# look into it later
pd.plotting.register_matplotlib_converters()
import numpy as np
import matplotlib.pyplot as plt
# look into it later
%matplotlib inline
import seaborn as sns
import numpy as np


# bmh makes more visual appealing
plt.style.use('bmh')

# set seed for reproducibility
np.random.seed(0)

# Load Data

In [20]:
# the test.csv file has no actual sales price for the competition purpose. So we need to split the trainning dataset into training and validation datasets
from sklearn.model_selection import train_test_split

# load the data
X = pd.read_csv('train.csv', index_col='Id')
X_test = pd.read_csv('test.csv', index_col='Id') # there's no target in here for the competition purpose

# remove rows with missing target – SalePrice
X.dropna(axis=0, subset=['SalePrice'], inplace=True)

# separate target from predictors
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

# print(X.columns == X_test.columns)
X.info()

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True]


# Preliminary Investigation

In [21]:
# shape of training data (num_rows, num_columns)
print(X.shape)

X.info()

# calculate percentage of missing value
total_cells = np.product(X.shape)
missing_value_cols = X.isnull().sum()
total_missing_cells = missing_value_cols.sum()
missing_percentage = total_missing_cells / total_cells * 100
print('Percentage of missing values: ' + str(format(missing_percentage, '.2f')) + '%')

(1460, 79)
<class 'pandas.core.frame.DataFrame'>
Index: 1460 entries, 1 to 1460
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  Ye

**Important:** Missing value is ~7%, so may be dropping all columns with missing value is the most effective strategy. It'll be proven below

In [27]:
# print('Columns with missing values:')
# print(missing_value_cols[missing_value_cols > 0])

# # numerical columns with missing value
# missing_value_num_cols = [col for col in X.columns if X[col].dtype != 'object' and X[col].isnull().sum() > 0]
# print('Numerical columns with missing value:')
# print(X[missing_value_num_cols].isnull().sum())
# print(len(missing_value_num_cols))

# missing_value_num_cols_test = [col for col in X_test.columns if X_test[col].dtype != 'object' and X_test[col].isnull().sum() > 0]
# print('Numerical columns with missing value:')
# print(X_test[missing_value_num_cols_test].isnull().sum())
# print(len(missing_value_num_cols_test))

# categorical columns with missing value
missing_value_cat_cols = [col for col in X.columns if X[col].dtype == 'object' and X[col].isnull().sum()]
print('Categorical columns with missing value:')
# print(X[missing_value_cat_cols].isnull().sum())
print(len(missing_value_cat_cols))

# categorical columns with missing value
missing_value_cat_cols_test = [col for col in X_test.columns if X_test[col].dtype == 'object' and X_test[col].isnull().sum()]
print('Categorical columns with missing value:')
# print(X_test[missing_value_cat_cols].isnull().sum())
print(len(missing_value_cat_cols_test))

Categorical columns with missing value:
16
Categorical columns with missing value:
0


# Handle Missing Value

## Step 1: Consider Dropping Columns with Missing Values

### Option 1: Drop **All** Columns

In [22]:
X_clean = X.dropna(axis=1)
X_test = X_test.dropna(axis=1)

# print(X_clean.columns == X_test.columns)

ValueError: Lengths must match to compare

### Option 2: Only Drop Columns with Many Missing Value, **the Rest Will Be Imputed**

In [7]:
threshold = 0.5
# calculate the percentage of missing values in each column
missing_percent = X.isnull().mean()
# Drop columns where the percentage of missing values is above the threshold
X_clean = X.loc[:, missing_percent < threshold]


### Step 2: Create a Pipeline

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

numerical_cols = [col for col in X_clean.columns if X_clean[col].dtype != 'object']
categorical_cols = [col for col in X_clean.columns if X_clean[col].dtype == 'object']
categorical_cols_test = [col for col in X_test.columns if X_test[col].dtype == 'object']

# preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='mean')

# preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    # ('imputer', SimpleImputer(strategy='most-frequent'),
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

### Step 3: Apply Cross-Validation

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

# X is broken into 5 folds (80% of X for training, 20% of X for validation, and do 5 interations)
cv_scores = cross_val_score(pipeline, X_clean, y, cv=5, scoring='neg_mean_absolute_error')

# Print the average cross-validation score
print('Mean Cross-Validation Score (MAE):', -cv_scores.mean())


Mean Cross-Validation Score (MAE): 17603.307196347036


**Results:** Confirming dropping all missing value produces best result.
- Option 1: Mean Cross-Validation Score (MAE): 17603.307196347036
- Option 2:
  - Mean Cross-Validation Score (MAE) with 'most frequent' imputation strategy for categorical data: 17610.4211369863
  - Mean Cross-Validation Score (MAE) with 'constant' imputation strategy for categorical data: 17615.257184931506

In [17]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
# OH_cols = pd.DataFrame(OH_encoder.fit_transform(X_clean[categorical_cols]))
OH_cols = OH_encoder.fit_transform(X_clean[categorical_cols])


# Get the names of the new columns
new_column_names = OH_encoder.get_feature_names_out(input_features=categorical_cols)
OH_cols = pd.DataFrame(OH_cols, columns=new_column_names)

# convert from float64 to int64
OH_cols = OH_cols.astype(int)

# One-hot encoding removed index; put it back
OH_cols.index = X_clean.index

# Remove categorical columns (will replace with one-hot encoding)
num_X = X_clean.drop(categorical_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X = pd.concat([num_X, OH_cols], axis=1)

# Ensure all columns' names have string type
OH_X.columns = OH_X.columns.astype(str)

print(OH_X.shape)
OH_X.columns

(1460, 215)


Index(['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       ...
       'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD',
       'SaleCondition_Abnorml', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=215)

# Feature Engineering

## Step 1. Calculate Mutual Information


In [10]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

# All discrete features should now have integer dtypes (double-check this before using MI!)
float_cols = [col for col in OH_X.columns if OH_X[col].dtype =='float64']
print(float_cols) # should be an empty list

discrete_features = OH_X.dtypes == int

mi_scores = make_mi_scores(OH_X, y, discrete_features)
mi_scores[::3]  # show a few features with their MI scores

[]


,MI Scores
OverallQual,0.571435
YearBuilt,0.409724
GarageCars,0.363921
YearRemodAdd,0.276464
TotRmsAbvGrd,0.216497
...,...
Heating_GasW,0.000000
Condition1_RRAn,0.000000
HouseStyle_2.5Unf,0.000000
RoofStyle_Gambrel,0.000000


In [11]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

In [12]:
threshold = 0.05
important_features = mi_scores[mi_scores > threshold]
# only keep features with mi score greater than threshold
important_X = OH_X[important_features.index]

important_X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1460 entries, 1 to 1460
Data columns (total 44 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   OverallQual            1460 non-null   int64
 1   GarageArea             1460 non-null   int64
 2   GrLivArea              1460 non-null   int64
 3   YearBuilt              1460 non-null   int64
 4   TotalBsmtSF            1460 non-null   int64
 5   LotArea                1460 non-null   int64
 6   GarageCars             1460 non-null   int64
 7   1stFlrSF               1460 non-null   int64
 8   MSSubClass             1460 non-null   int64
 9   YearRemodAdd           1460 non-null   int64
 10  FullBath               1460 non-null   int64
 11  ExterQual_TA           1460 non-null   int64
 12  TotRmsAbvGrd           1460 non-null   int64
 13  KitchenQual_TA         1460 non-null   int64
 14  ExterQual_Gd           1460 non-null   int64
 15  OpenPorchSF            1460 non-null   int6

## Step 2. Discover Interaction

In [13]:
merged_X = pd.concat([important_X, y], axis=1)

In [22]:
# experiment with GrLivArea and Fireplaces
# feature = 'GrLivArea'
# sns.regplot(x=feature, y='SalePrice', data=merged_X, scatter_kws={'alpha': 0.5})

# sns.lmplot(
#     x=feature, y='SalePrice', hue='Fireplaces', col='Fireplaces',
#     data=merged_X, scatter_kws={"edgecolor": 'w'}, col_wrap=2, height=3,
# );

# new features. I use merge_X instead of important_X as I sometimes need visualizations between features and SalePrice like above
important_X['GrLivArea_Fireplaces'] = important_X['GrLivArea'] * important_X['Fireplaces']
# important_X['TotalSF_NEW'] = important_X['TotalBsmtSF'] + important_X['GrLivArea']
# important_X['TotalBath_NEW'] = important_X['FullBath'] + 0.5 * important_X['HalfBath'] # + important_X['BsmtFullBath'] + 0.5 * important_X['BsmtHalfBath']
# important_X['TotalOutdoorSF_NEW'] = important_X['WoodDeckSF'] + important_X['OpenPorchSF'] # + important_X['3SsnPorch'] + important_X['EnclosedPorch'] + important_X['ScreenPorch']
# important_X['TotalLivingAreaRatio_NEW'] = merged_X['GrLivArea'] / important_X['LotArea']

# important_X.drop(columns=['TotalSF_NEW'])
# important_X.drop(columns=['TotalBath_NEW'])
# important_X.drop(columns=['TotalOutdoorSF_NEW'])
# important_X.drop(columns=['TotalLivingAreaRatio_NEW'])
# Add the new feature to the important_X DataFrame
# important_X['GrLivArea_Fireplaces'] = merged_X['GrLivArea_Fireplaces']

# Update discrete_features to include the new feature
discrete_features = important_X.dtypes == int

# Calculate the MI score for the new feature set, including the interaction term
new_feature_mi_scores = make_mi_scores(important_X, y, discrete_features)
# print(new_feature_mi_scores)

# print(new_feature_mi_scores.loc['GrLivArea_Fireplaces'])
# print(new_feature_mi_scores.loc['Fireplaces'])

# feature = 'GrLivArea_Fireplaces'
# sns.regplot(x=feature, y='SalePrice', data=merged_X, scatter_kws={'alpha': 0.5})

<ipython-input-22-c4b092412581>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  important_X['GrLivArea_Fireplaces'] = important_X['GrLivArea'] * important_X['Fireplaces']
<ipython-input-22-c4b092412581>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  important_X['TotalSF_NEW'] = important_X['TotalBsmtSF'] + important_X['GrLivArea']
<ipython-input-22-c4b092412581>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

**Evaluate new feature:**

In [16]:
from xgboost import XGBRegressor
# Create an instance of the XGBRegressor
model = XGBRegressor(n_estimators=500, learning_rate=0.05, random_state=0)
# model = RandomForestRegressor(n_estimators=100, random_state=0)

# set up the pipeline (no preprocessing needed as it has already been handled)
pipeline = Pipeline(steps=[
    ('model', model)
])

# perform Cross-Validation without the new feature
X_without_new_feature = merged_X.drop(columns=['SalePrice'])
cv_scores_without_new = cross_val_score(pipeline, X_without_new_feature, y, cv=5, scoring='neg_mean_absolute_error')
print('Mean Cross-Validation Score (MAE) without new feature:', -cv_scores_without_new.mean())

Mean Cross-Validation Score (MAE) without new feature: 16977.57307630565


In [23]:
# Step 5: Perform Cross-Validation with the new feature
cv_scores_with_new = cross_val_score(pipeline, important_X, y, cv=5, scoring='neg_mean_absolute_error')

# Step 6: Compare the average MAE scores

print('Mean Cross-Validation Score (MAE) with new feature:', -cv_scores_with_new.mean())

Mean Cross-Validation Score (MAE) with new feature: 16994.568530607878


**Results:**

- Mean Cross-Validation Score (MAE) without new feature: 16977.57307630565
- Mean Cross-Validation Score (MAE) with new feature: (1) 16610.54620344606; (2) 16794.879559075343; (3) 16909.589816994863; (4) 16994.568530607878

# Make Prediction

In [18]:
pipeline.fit(important_X, y)

X_test.dropna(axis=1)
X_test_encoded = pd.get_dummies(X_test, drop_first=True)

# make predictions using the pipeline
test_preds = pipeline.predict(X_test)

# create a DataFrame for the predictions
# predictions_df = pd.DataFrame(predictions, columns=['SalePrice'])

# If you have an ID column in your test dataset, you can also add it to the predictions DataFrame
# predictions_df['Id'] = test_ids  # Replace test_ids with your actual ID column

output = pd.DataFrame({'Id': X_test.Id,
                       'SalePrice': test_preds})
output.to_csv('submission.csv', index=False)


NameError: name 'important_X' is not defined